<a href="https://www.kaggle.com/code/j13mehul/dl-architecture-10-features-bert-embedding?scriptVersionId=143536186" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="background-color: powderblue; padding: 10px; border-radius: 15px;">
    <h1 style="text-align: center; font-size: 28px;">🏁 Setting the stage 🎌</h1>
</div>

In [ ]:
!pip install "/kaggle/input/autocorrect/autocorrect-2.6.1.tar"
!pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np 
import pandas as pd
import string
from collections import Counter
import re
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from autocorrect import Speller
from spellchecker import SpellChecker

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

import optuna
import lightgbm as lgb

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch import nn
import torch.optim as optim
from transformers import DataCollatorWithPadding, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments

In [ ]:
#Train
prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
df = prompts.merge(summaries, on = ['prompt_id'],how ='left')

#Test
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test = prompts_test.merge(summaries_test, on = ['prompt_id'],how ='left')

<div style="background-color: #c4affa; padding: 10px; border-radius: 15px;">
    <h1 style="text-align: center; font-size: 28px;">🧹 Data Cleaning & Processing ⚙️</h1>
</div>

In [ ]:
spell = Speller(lang='en')

df['correct_text'] = df['text'].apply(lambda x: "".join([spell(i) for i in x]))
test['correct_text'] = test['text'].apply(lambda x: "".join([spell(i) for i in x]))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text


df['correct_text'] = df['text'].apply(lambda x: "".join([spell(i) for i in x]))
test['correct_text'] = test['text'].apply(lambda x: "".join([spell(i) for i in x]))

df['prompt_text'] = df['prompt_text'].apply(lambda x: "".join([spell(i) for i in x]))
test['prompt_text'] = test['prompt_text'].apply(lambda x: "".join([spell(i) for i in x]))

<div style="background-color: #ea9bc4; padding: 10px; border-radius: 15px;">
    <h1 style="text-align: center; font-size: 28px;">  🧑‍🔬 Feature Engineering 👨‍🔬</h1>
</div>

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 0</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Deberta V3 predictions</h1></div>

In [ ]:
model_name = "/kaggle/input/deberta-v3-base/deberta-v3-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, problem_type="regression")

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, has_labels=True):
        self.df = df
        self.prompt_titles = df["question"].values.tolist()
        self.texts = df["correct_text"].values.tolist()
        self.encoded_examples = tokenizer(
            text=self.prompt_titles,
            text_pair=self.texts,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        )
        self.has_labels = has_labels
        
        if self.has_labels:
            self.labels_list = df[["content", "wording"]].values.tolist()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = {
            "input_ids": self.encoded_examples["input_ids"][idx],
            "attention_mask": self.encoded_examples["attention_mask"][idx],
            "token_type_ids": self.encoded_examples["token_type_ids"][idx]
        }
        
        if self.has_labels:
            item["labels"] = torch.tensor(self.labels_list[idx])
        
        return item

In [ ]:
def compute_mcrmse(eval_pred):

    predictions, labels = eval_pred
    squared_errors = np.square(predictions - labels)
    mean_squared_errors = np.mean(squared_errors, axis=0)
    
    rmse = np.sqrt(mean_squared_errors)
    
    mcrmse_value = np.mean(rmse)
    
    content_rmse = rmse[0]
    wording_rmse = rmse[1]
    
    return {
        "mcrmse": mcrmse_value,
        "content_rmse": content_rmse,
        "wording_rmse": wording_rmse
    }

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

df['question'] = df["prompt_title"] + "\n" + df["prompt_question"] + "\n" + df["prompt_text"]
test['question'] = test["prompt_title"] + "\n" + test["prompt_question"] + "\n" + test["prompt_text"]

df_train, df_valid = train_test_split(df, test_size=0.2, random_state=2023)

train_dataset = CustomDataset(df_train)
valid_dataset = CustomDataset(df_valid)
test_dataset  = CustomDataset(test, has_labels=False)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="output",             
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=4,    
    learning_rate=1e-3,            
    lr_scheduler_type="linear",      
    warmup_ratio=0.01,               
    num_train_epochs=3,              
    save_strategy="epoch",           
    logging_strategy="epoch",        
    evaluation_strategy="epoch",     
    load_best_model_at_end=True,     
    metric_for_best_model="mcrmse",           
    fp16=False,                      
    report_to='none',
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_mcrmse,
)
trainer.train()

trainer.save_model("best_model")

df_train = df_train.reset_index(drop = True)
df_valid = df_valid.reset_index(drop = True)

predictions_train = trainer.predict(train_dataset)
deberta_content = predictions_train.predictions[:, 0].tolist()
deberta_wording = predictions_train.predictions[:, 1].tolist()

df_train['deberta_content'] = deberta_content
df_train['deberta_wording'] = deberta_wording

predictions_val = trainer.predict(valid_dataset)
deberta_content = predictions_val.predictions[:, 0].tolist()
deberta_wording = predictions_val.predictions[:, 1].tolist()

df_valid['deberta_content'] = deberta_content
df_valid['deberta_wording'] = deberta_wording

predictions_test = trainer.predict(test_dataset)
deberta_content = predictions_test.predictions[:, 0].tolist()
deberta_wording = predictions_test.predictions[:, 1].tolist()

test['deberta_content'] = deberta_content
test['deberta_wording'] = deberta_wording

In [ ]:
df_train_val = pd.concat([df_train[['prompt_id','student_id','deberta_content','deberta_wording']], 
                          df_valid[['prompt_id','student_id','deberta_content','deberta_wording']]], ignore_index=True)

df = df.merge(df_train_val, on = ['prompt_id','student_id'], how = 'left')

In [ ]:
del df_train_val, trainer, train_dataset, valid_dataset, test_dataset, data_collator, df_train, df_valid, predictions_train, predictions_val, predictions_test
gc.collect()

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 1</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Ratio of No of words in Student's response vs context</h1></div>

In [ ]:
def wordsRatio(context, response):
    context = context.split()
    response = response.split()
    return len(response)/len(context)

df["word_count_ratio"] = df.apply(lambda x: wordsRatio(x['prompt_text'], x['correct_text']), axis=1)
test["word_count_ratio"] = test.apply(lambda x: wordsRatio(x['prompt_text'], x['correct_text']), axis=1)

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 2</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Number of spelling errors in Student's summary</h1></div>

In [ ]:
def misspelledRatio(x):
    spell = SpellChecker()
    words = x.split()
    misspelled = spell.unknown(words)
    return len(misspelled)/len(words)

df["misspelled_ratio"] = df['text'].apply(lambda x: misspelledRatio(x))  
test["misspelled_ratio"] = test['text'].apply(lambda x: misspelledRatio(x))

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 3</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Stop word Ratio in Student's summary</h1></div>

In [ ]:
stop_words = stopwords.words('english')
def StopwordsRatio(context, response):
    length  = len(response.split())
    response = " ".join([i for i in response.split() if i in stop_words]) 
    return len(response)/length

df["stop_word_ratio"] = df.apply(lambda x: StopwordsRatio(x['prompt_text'], x['correct_text']), axis=1)
test["stop_word_ratio"] = test.apply(lambda x: StopwordsRatio(x['prompt_text'], x['correct_text']), axis=1)

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 4</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Count of same words in Student's summary</h1></div>

In [ ]:
def sameWordsRatio(context, response):
    context = " ".join([i for i in context.split() if i not in stop_words]) 
    response = " ".join([i for i in response.split() if i not in stop_words]) 
    return len(set(response).intersection(set(context)))/len(response)

df["same_word_ratio"] = df.apply(lambda x: sameWordsRatio(x['prompt_text'], x['correct_text']), axis=1)
test["same_word_ratio"] = test.apply(lambda x: sameWordsRatio(x['prompt_text'], x['correct_text']), axis=1)

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 5</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Average reading speed in words per minute </h1></div>

In [ ]:
def readTime(x):
    average_wpm = 200
    word_count = len(x.split())
    return word_count / average_wpm

df["read_time"] = df['text'].apply(lambda x: readTime(x))  
test["read_time"] = test['text'].apply(lambda x: readTime(x))  

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 6</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Diversity of words in the Student's response </h1></div>

In [ ]:
def diversityIndex(x):
    words = x.split()
    word_counts = Counter(words)
    total_words = len(words)
    return 1 - sum((count / total_words) ** 2 for count in word_counts.values())

df["diversity_index"] = df['text'].apply(lambda x: diversityIndex(x))  
test["diversity_index"] = test['text'].apply(lambda x: diversityIndex(x))  

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 7</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> TF IDF to identify most frequent words in context and response </h1></div>

In [ ]:
def tfidfImportance(context, response):
    top_n = 10

    context = " ".join([i for i in context.split() if i not in stop_words]) 
    response = " ".join([i for i in response.split() if i not in stop_words]) 
    
    corpus = [
        context, response
     ]

    tfidf_unigram = TfidfVectorizer(ngram_range=(1, 1))
    tfidf_unigram_matrix = tfidf_unigram.fit_transform(corpus)
    unigram_feature_names = tfidf_unigram.get_feature_names_out()
    dense_unigram_array = tfidf_unigram_matrix.toarray()
    unigram_df = pd.DataFrame(data=dense_unigram_array, columns=unigram_feature_names)

    top_unigrams = []
    for index, row in unigram_df.iterrows():
        top_unigrams.append(list(row.nlargest(top_n).index))

    tfidf_bigram = TfidfVectorizer(ngram_range=(2, 2)) 

    tfidf_bigram_matrix = tfidf_bigram.fit_transform(corpus)
    bigram_feature_names = tfidf_bigram.get_feature_names_out()
    dense_bigram_array = tfidf_bigram_matrix.toarray()
    bigram_df = pd.DataFrame(data=dense_bigram_array, columns=bigram_feature_names)

    top_bigrams = []
    for index, row in bigram_df.iterrows():
        top_bigrams.append(list(row.nlargest(top_n).index))


    return len(set(top_unigrams[0]).intersection(set(top_unigrams[1])))/top_n, len(set(top_bigrams[0]).intersection(set(top_bigrams[1])))/top_n

df[['top_unigrams_ratio','top_bigrams_ratio']]= df.apply(lambda x: tfidfImportance(x['prompt_text'], x['correct_text']), axis=1, result_type ='expand')
test[['top_unigrams_ratio','top_bigrams_ratio']]= test.apply(lambda x: tfidfImportance(x['prompt_text'], x['correct_text']), axis=1, result_type ='expand')

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 8</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Cosine similarity of context and response using TD IDF vectors</h1></div>

In [ ]:
def tdidfSimilarity(context, response):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([context, response])
    return cosine_similarity(vectors[0:1], vectors[1:2])[0][0] 

df['tfidf_similarity']= df.apply(lambda x: tdidfSimilarity(x['prompt_text'], x['correct_text']), axis=1)
test['tfidf_similarity']= test.apply(lambda x: tdidfSimilarity(x['prompt_text'], x['correct_text']), axis=1)

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 9</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> N Grams Co occurence context and response </h1></div>

In [ ]:
def tokenize(sentence):
    sentence = re.sub(r'[^\w\s]', '', sentence.lower())
    tokens = sentence.split()
    return tokens

def generate_ngrams(tokens, n):
    ngrams = []
    for i in range(len(tokens) - n + 1):
        ngram = tuple(tokens[i:i + n])
        ngrams.append(ngram)
    return ngrams


def cooccurrenceRatio(context, response, n_gram_size = 2):
    tokens1 = tokenize(context)
    tokens2 = tokenize(response)

    ngrams1 = generate_ngrams(tokens1, n_gram_size)
    ngrams2 = generate_ngrams(tokens2, n_gram_size)

    co_occurrence_count = len(set(ngrams1) & set(ngrams2))
    
    return co_occurrence_count/(min(len(ngrams1),len(ngrams2))+1)

df['bi_gram_ratio'] = df.apply(lambda x: cooccurrenceRatio(x['prompt_text'], x['correct_text']), axis=1)
test['bi_gram_ratio'] = test.apply(lambda x: cooccurrenceRatio(x['prompt_text'], x['correct_text']), axis=1)

df['tri_gram_ratio'] = df.apply(lambda x: cooccurrenceRatio(x['prompt_text'], x['correct_text'], n_gram_size = 3), axis=1)
test['tri_gram_ratio'] = test.apply(lambda x: cooccurrenceRatio(x['prompt_text'], x['correct_text'],n_gram_size = 3 ), axis=1)

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 10</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Check punctuation in Student's response </h1></div>

In [ ]:
def checkPunctuations(x):
    punctuation_count = 0

    for char in x:
        if char in string.punctuation:
            punctuation_count += 1
    return punctuation_count/len(x.split())

df["punctuation_ratio"] = df['text'].apply(lambda x: checkPunctuations(x))
test["punctuation_ratio"] = test['text'].apply(lambda x: checkPunctuations(x)) 

<div><h1 style="text-align: left; font-size: 25px;"> 📍 Feature 11</h1></div>
<div><h1 style="text-align: left; font-size: 20px;"> Identifying Co occurences of NER in context and response </h1></div>

In [ ]:
nlp = spacy.load("en_core_web_sm")

def nerCooccurrenceRatio(context, response, n_gram_size = 2):
    
    doc1 = nlp(context)
    doc2 = nlp(response)
    
    entities1 = {ent.text for ent in doc1.ents}
    entities2 = {ent.text for ent in doc2.ents}
    
    overlap_entities = entities1.intersection(entities2)
    return len(overlap_entities)/(min(len(entities1),len(entities2))+1)


df['nercooccurrence_ratio'] = df.apply(lambda x: nerCooccurrenceRatio(x['prompt_text'], x['correct_text']), axis=1)
test['nercooccurrence_ratio'] = test.apply(lambda x: nerCooccurrenceRatio(x['prompt_text'], x['correct_text']), axis=1)

<div style="background-color: #b6d7a8; padding: 10px; border-radius: 15px;">
    <h1 style="text-align: center; font-size: 28px;"> 🔬 EDA ⚗️</h1>
</div>

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1,2, 1)  
sns.histplot(x=df['content'], bins=50, kde=True, color='#ed8240')
plt.title(f"Histogram of content")

plt.subplot(1,2, 2)  
sns.histplot(x=df['wording'], bins=50, kde=True, color='#c540ed')
plt.title(f"Histogram of wording")

plt.tight_layout() 
plt.show()

In [ ]:
cols = [
    'deberta_content',
    'deberta_wording',
    'word_count_ratio', 
    'misspelled_ratio', 
    'stop_word_ratio',
    'same_word_ratio', 
    'read_time', 
    'diversity_index', 
    'top_unigrams_ratio',
    'top_bigrams_ratio',
    'tfidf_similarity',
    'punctuation_ratio', 
    'bi_gram_ratio',
    'tri_gram_ratio', 
    'nercooccurrence_ratio'
]

In [ ]:
all_cols = ['content','wording'] + cols
plt.figure(figsize=(15, 7))
sns.heatmap(df[all_cols].corr(), annot=True)

In [ ]:
for col in cols:
    print(f'Plots for {col}')
    plt.figure(figsize=(15, 5))

    plt.subplot(1,3, 1)  
    sns.scatterplot(x=df[col], y=df['content'], color='#4082ed')
    plt.title("Scatterplot with content")

    plt.subplot(1, 3, 2)  
    sns.scatterplot(x=df[col], y=df['wording'], color='#40b9ed')
    plt.title("Scatterplot with wording")

    plt.subplot(1, 3, 3)  
    sns.histplot(x=df[col], bins=50, kde=True, color='#40d3ed')
    plt.title(f"Histogram of {col}")

    plt.tight_layout() 
    plt.show()


<div style="background-color: #8dc1e9; padding: 10px; border-radius: 15px;">
    <h1 style="text-align: center; font-size: 28px;"> 🐥 Bert Embedding 🐥</h1>
</div>

In [ ]:
gc.collect()

In [ ]:
model_name = "/kaggle/input/huggingface-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

max_chunk_length = 512


def get_embeddings_for_sent(sent):
    input_ids = tokenizer(sent, padding=True, truncation=True, max_length=max_chunk_length, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**input_ids)
    cls_embedding = outputs.last_hidden_state[:, 0, :] 
    return cls_embedding


def get_embeddings_for_para(document):
    chunks = [document[i:i + max_chunk_length] for i in range(0, len(document), max_chunk_length)]
    chunk_embeddings = []
    for chunk in chunks:
        input_ids = tokenizer(chunk, padding=True, truncation=True, max_length=max_chunk_length, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**input_ids)
            cls_embedding = outputs.last_hidden_state[:, 0, :]
        chunk_embeddings.append(cls_embedding)
        final_embedding = torch.mean(torch.stack(chunk_embeddings), dim=0)
        return final_embedding

In [ ]:
# Embedding of prompt_question 
prmt_data = df[['prompt_id','prompt_question','prompt_text']].drop_duplicates().reset_index(drop = True)
prmt_test = test[['prompt_id','prompt_question','prompt_text']].drop_duplicates().reset_index(drop = True)

prmt_data["text_emb"] = prmt_data['prompt_question'].apply(lambda x: get_embeddings_for_sent(x))  
prmt_data["flattened_embedding"] = prmt_data["text_emb"].apply(lambda x: x.flatten().numpy())
emb_df = pd.DataFrame(prmt_data["flattened_embedding"].to_list())
emb_df = pd.concat([prmt_data[['prompt_id']], emb_df],axis = 1)
emb_df = df[['prompt_id']].merge(emb_df, on = 'prompt_id', how = 'left').drop(['prompt_id'], axis= 1)

prmt_test["text_emb"] = prmt_test['prompt_question'].apply(lambda x: get_embeddings_for_sent(x))  
prmt_test["flattened_embedding"] = prmt_test["text_emb"].apply(lambda x: x.flatten().numpy())
emb_test = pd.DataFrame(prmt_test["flattened_embedding"].to_list())
emb_test = pd.concat([prmt_test[['prompt_id']], emb_test],axis = 1)
emb_test = test[['prompt_id']].merge(emb_test, on = 'prompt_id', how = 'left').drop(['prompt_id'],axis= 1)

In [ ]:
# Embedding of prompt_text
prmt_data = df[['prompt_id','prompt_question','prompt_text']].drop_duplicates().reset_index(drop = True)
prmt_test = test[['prompt_id','prompt_question','prompt_text']].drop_duplicates().reset_index(drop = True)

prmt_data["text_emb"] = prmt_data['prompt_text'].apply(lambda x: get_embeddings_for_para(x))  
prmt_data["flattened_embedding"] = prmt_data["text_emb"].apply(lambda x: x.flatten().numpy())
emb_cntxt_df = pd.DataFrame(prmt_data["flattened_embedding"].to_list())
emb_cntxt_df = pd.concat([prmt_data[['prompt_id']], emb_cntxt_df],axis = 1)
emb_cntxt_df = df[['prompt_id']].merge(emb_cntxt_df, on = 'prompt_id', how = 'left').drop(['prompt_id'], axis= 1)

prmt_test["text_emb"] = prmt_test['prompt_text'].apply(lambda x: get_embeddings_for_para(x))  
prmt_test["flattened_embedding"] = prmt_test["text_emb"].apply(lambda x: x.flatten().numpy())
emb_cntxt_test = pd.DataFrame(prmt_test["flattened_embedding"].to_list())
emb_cntxt_test = pd.concat([prmt_test[['prompt_id']], emb_cntxt_test],axis = 1)
emb_cntxt_test = test[['prompt_id']].merge(emb_cntxt_test, on = 'prompt_id', how = 'left').drop(['prompt_id'],axis= 1)

In [ ]:
# Embedding of response
df["correct_text_emb"] = df['correct_text'].apply(lambda x: get_embeddings_for_sent(x))  
df["correct_text_flattened_embedding"] = df["correct_text_emb"].apply(lambda x: x.flatten().numpy())
emb_corrt_df = pd.DataFrame(df["correct_text_flattened_embedding"].to_list())

test["correct_text_emb"] = test['correct_text'].apply(lambda x: get_embeddings_for_sent(x))  
test["correct_text_flattened_embedding"] = test["correct_text_emb"].apply(lambda x: x.flatten().numpy())
emb_corrt_test = pd.DataFrame(test["correct_text_flattened_embedding"].to_list())

In [ ]:
train = df[cols]
test_score = test[cols]
Y = df[['content','wording']]

In [ ]:
list_similarity = []
for index, row in emb_cntxt_df.iterrows():
    x = [row.to_numpy()]
    y = [emb_corrt_df.iloc[index,:].to_numpy()]
    
    x = x / np.linalg.norm(x, axis=1, keepdims=True)
    y = y / np.linalg.norm(y, axis=1, keepdims=True)
    list_similarity.append(cosine_similarity(x, y)[0][0])

In [ ]:
list_similarity_test = []
for index, row in emb_cntxt_test.iterrows():
    x = [row.to_numpy()]
    y = [emb_corrt_test.iloc[index,:].to_numpy()]
    
    x = x / np.linalg.norm(x, axis=1, keepdims=True)
    y = y / np.linalg.norm(y, axis=1, keepdims=True)
    list_similarity_test.append(cosine_similarity(x, y)[0][0])

In [ ]:
train['cosine_similarity'] =  list_similarity
test_score['cosine_similarity'] =  list_similarity_test

In [ ]:
X_train, X_val = train_test_split(train, test_size=0.1)

In [ ]:
emb_cntxt_train = emb_cntxt_df.filter(X_train.index, axis=0)
emb_cntxt_val = emb_cntxt_df.filter(X_val.index, axis=0)

emb_corrt_train = emb_corrt_df.filter(X_train.index, axis=0)
emb_corrt_val = emb_corrt_df.filter(X_val.index, axis=0)

emb_train = emb_df.filter(X_train.index, axis=0)
emb_val = emb_df.filter(X_val.index, axis=0)

Y_train = Y.filter(X_train.index, axis=0)
Y_val = Y.filter(X_val.index, axis=0)

In [ ]:
# for question
num_components = 3
q_label = ['q'+ str(i) for i in range(num_components)]
pca = PCA(n_components=num_components)

emb_train_pca = pca.fit_transform(emb_train)
emb_val_pca = pca.transform(emb_val)
emb_test_pca = pca.transform(emb_test)

principal_components = pca.components_
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained Variance Ratio:", sum(explained_variance_ratio))

In [ ]:
# for response
num_components = 10
r_label = ['r'+ str(i) for i in range(num_components)]
pca = PCA(n_components=num_components)

emb_cntxt_train_pca = pca.fit_transform(emb_cntxt_train)
emb_cntxt_val_pca = pca.transform(emb_cntxt_val)
emb_cntxt_test_pca = pca.transform(emb_cntxt_test)

principal_components = pca.components_
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained Variance Ratio:", sum(explained_variance_ratio))

In [ ]:
# for context
num_components = 10
c_label = ['c'+ str(i) for i in range(num_components)]

pca = PCA(n_components=num_components)

emb_corrt_train_pca = pca.fit_transform(emb_corrt_train)
emb_corrt_val_pca = pca.transform(emb_corrt_val)
emb_corrt_test_pca = pca.transform(emb_corrt_test)

principal_components = pca.components_
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained Variance Ratio:", sum(explained_variance_ratio))

In [ ]:
feat_cols = list(X_train.columns) + q_label + r_label + c_label

In [ ]:
X_train = X_train.reset_index(drop = True)
X_val = X_val.reset_index(drop = True)
test_score = test_score.reset_index(drop = True)

In [ ]:
X_train_gbm = pd.concat([X_train, pd.DataFrame(emb_train_pca), pd.DataFrame(emb_cntxt_train_pca), pd.DataFrame(emb_corrt_train_pca)],axis = 1)
X_val_gbm = pd.concat([X_val, pd.DataFrame(emb_val_pca), pd.DataFrame(emb_cntxt_val_pca), pd.DataFrame(emb_corrt_val_pca)],axis = 1)
X_test_gbm = pd.concat([test_score, pd.DataFrame(emb_test_pca), pd.DataFrame(emb_cntxt_test_pca), pd.DataFrame(emb_corrt_test_pca)],axis = 1)

In [ ]:
X_train_gbm.columns = feat_cols
X_val_gbm.columns = feat_cols
X_test_gbm.columns = feat_cols

In [ ]:
gc.collect()

In [ ]:
# LightGBM

import logging
null_handler = logging.NullHandler()
optuna.logging.get_logger("optuna").addHandler(null_handler)
n_trials = 50

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.7, 0.99),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 0.99),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.01, 10),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.01, 10),
        'verbose' : -1
    }
    
    train_data = lgb.Dataset(X_train_gbm, label=Y_train[Y])
    valid_data = lgb.Dataset(X_val_gbm, label=Y_val[Y], reference=train_data)
    
    model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], num_boost_round=10000, early_stopping_rounds=20, verbose_eval=False)
    
    valid_preds = model.predict(X_val_gbm)
    rmse = np.sqrt(mean_squared_error(Y_val[Y], valid_preds))
    
    return rmse


print('Train for content')
Y = 'content'
study1 = optuna.create_study(direction='minimize')
study1.optimize(objective, n_trials=n_trials)

best_trial = study1.best_trial
print("Best RMSE: {:.4f}".format(best_trial.value))
print("Best Params:", best_trial.params)


best_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': best_trial.params['num_leaves'],  
    'learning_rate': best_trial.params['learning_rate'],
    'feature_fraction': best_trial.params['feature_fraction'],
    'bagging_fraction': best_trial.params['bagging_fraction'],
    'bagging_freq': best_trial.params['bagging_freq'],
    'min_child_samples': best_trial.params['min_child_samples'],
    'lambda_l1': best_trial.params['lambda_l1'],
    'lambda_l2': best_trial.params['lambda_l2'],
    'verbose' : -1
}


train_data = lgb.Dataset(X_train_gbm, label=Y_train[Y])
best_model = lgb.train(best_params, train_data, num_boost_round=1000)
content_list = best_model.predict(X_test_gbm)

print('Train for wording')
Y = 'wording'
study2 = optuna.create_study(direction='minimize')
study2.optimize(objective, n_trials=n_trials)

best_trial = study2.best_trial
print("Best RMSE: {:.4f}".format(best_trial.value))
print("Best Params:", best_trial.params)

best_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': best_trial.params['num_leaves'],  
    'learning_rate': best_trial.params['learning_rate'],
    'feature_fraction': best_trial.params['feature_fraction'],
    'bagging_fraction': best_trial.params['bagging_fraction'],
    'bagging_freq': best_trial.params['bagging_freq'],
    'min_child_samples': best_trial.params['min_child_samples'],
    'lambda_l1': best_trial.params['lambda_l1'],
    'lambda_l2': best_trial.params['lambda_l2'],
    'verbose' : -1
}


train_data = lgb.Dataset(X_train_gbm, label=Y_train[Y])
best_model = lgb.train(best_params, train_data, num_boost_round=1000)
wording_list = best_model.predict(X_test_gbm)

In [ ]:
# num_side_features = len(X_train.columns)
# hidden_dim1 = int(num_side_features/2)
# num_emb= 768

In [ ]:
# batch_size = 32

# question_inputs_train = torch.from_numpy(emb_cntxt_train.to_numpy())
# context_inputs_train = torch.from_numpy(emb_train.to_numpy())
# response_inputs_train = torch.from_numpy(emb_corrt_train.to_numpy())
# side_feat_train = torch.from_numpy(X_train.to_numpy().astype(np.float32))
# Y_train = torch.from_numpy(Y_train.to_numpy())

# question_inputs_val = torch.from_numpy(emb_cntxt_val.to_numpy())
# context_inputs_val = torch.from_numpy(emb_val.to_numpy())
# response_inputs_val = torch.from_numpy(emb_corrt_val.to_numpy())
# side_feat_val = torch.from_numpy(X_val.to_numpy().astype(np.float32))
# Y_val = torch.from_numpy(Y_val.to_numpy())

# question_inputs_test = torch.from_numpy(emb_cntxt_test.to_numpy())
# context_inputs_test = torch.from_numpy(emb_test.to_numpy())
# response_inputs_test = torch.from_numpy(emb_corrt_test.to_numpy())
# side_feat_test = torch.from_numpy(test_score.to_numpy().astype(np.float32))

# dataset_tensor_train = TensorDataset(question_inputs_train, context_inputs_train, response_inputs_train, side_feat_train, Y_train)
# dataset_tensor_val = TensorDataset(question_inputs_val, context_inputs_val, response_inputs_val, side_feat_val, Y_val)

# train_loader = DataLoader(dataset_tensor_train, batch_size, shuffle=False)  
# val_loader = DataLoader(dataset_tensor_val, batch_size)

<div style="background-color: #ffe599; padding: 10px; border-radius: 15px;">
    <h1 style="text-align: center; font-size: 28px;"> 🏭 Deep learning Architecture 🏗️</h1>
</div>

In [ ]:
# # Define the deep learning model
# class BertDotProductModel(nn.Module):
#     def __init__(self, num_emb, num_side_features, hidden_dim1):
#         super(BertDotProductModel, self).__init__()

#         self.side_feature_layer = nn.Linear(num_side_features, hidden_dim1)  
        
#         self.fc1 = nn.Linear((num_emb * 2 + hidden_dim1 + 1), num_emb)
#         self.fc2 = nn.Linear(768, 512) 
#         self.fc3 = nn.Linear(512, 512) 
#         self.fc4 = nn.Linear(512, 256) 
#         self.fc5 = nn.Linear(256, 128) 
#         self.fc6 = nn.Linear(128, 64)
#         self.fc7 = nn.Linear(64, 2) 
#         self.relu = torch.nn.ReLU()

#     def forward(self, question, context, response, side_features):

#         dot_product = torch.sum(context * response, dim=1)
#         side_features = self.side_feature_layer(side_features)
#         residual_context = torch.sub(question, context)
#         residual_response = torch.sub(question, response)
#         concat_features = torch.cat((dot_product.unsqueeze(1), residual_context, residual_response, side_features), dim=1)

#         fc_output = self.relu(concat_features)
#         fc_output = self.fc1(fc_output)
#         fc_output = self.relu(fc_output)
#         fc_output = self.fc2(fc_output)
#         fc_output = self.relu(fc_output)
#         fc_output = self.fc3(fc_output)
#         fc_output = self.relu(fc_output)
#         fc_output = self.fc4(fc_output)
#         fc_output = self.relu(fc_output)
#         fc_output = self.fc5(fc_output)
#         fc_output = self.relu(fc_output)
#         fc_output = self.fc6(fc_output)
#         fc_output = self.relu(fc_output)
#         fc_output = self.fc7(fc_output)
        
#         return fc_output

In [ ]:
# model = BertDotProductModel(num_emb, num_side_features, hidden_dim1)

In [ ]:
# training_loss = []
# validation_loss = []

# criterion = nn.MSELoss()

# optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# num_epochs = 20

# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0.0

#     for question, context, response, side_features, target in train_loader:
#         optimizer.zero_grad()
#         outputs = model(question, context, response, side_features)  
    
#         target = target.to(torch.float)

#         loss1 = torch.sqrt(criterion(outputs[:, 0].squeeze(), target[:,0]))
#         loss2 = torch.sqrt(criterion(outputs[:, 1].squeeze(), target[:,1])) 
        
#         loss = loss1 + loss2
#         total_loss += loss.item()
        
#         loss.backward()
#         optimizer.step()
        
#     training_loss.append(total_loss / len(train_loader))

#     model.eval()
#     valid_loss = 0.0

#     with torch.no_grad():
#         for question, context, response, side_features, target in val_loader:
#             outputs = model(question, context, response, side_features)
#             loss1 = torch.sqrt(criterion(outputs[:, 0].squeeze(), target[:,0])) 
#             loss2 = torch.sqrt(criterion(outputs[:, 1].squeeze(), target[:,1])) 
            
#             loss = loss1 + loss2 
            
#             valid_loss += loss.item()
            
#     validation_loss.append(valid_loss / len(val_loader))

#     scheduler.step()

In [ ]:
# sns.lineplot(x=[i for i in range(num_epochs)], y=training_loss)
# sns.lineplot(x=[i for i in range(num_epochs)], y=validation_loss)

In [ ]:
# model.eval()

# with torch.no_grad():
#     outputs = model(question_inputs_test, context_inputs_test, response_inputs_test, side_feat_test)

# outputs = outputs.numpy()

In [ ]:

# content_list = outputs[:, 0].tolist()
# wording_list = outputs[:, 1].tolist()

submission = test[['student_id']]
submission['content'] = content_list
submission['wording'] = wording_list

submission.to_csv('submission.csv', index=False)

submission.head()

<div style="background-color: #bcbcbc; padding: 10px; border-radius: 15px;">
    <h1 style="text-align: center; font-size: 28px;"> If you find it useful, please upvote for my efforts ⬆️ </h1>
</div>